In [46]:
import io
import json
import flask
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torchvision.models import resnet50
from torch.autograd import Variable

使用时需要更改的三个东西
1. 模型
2. 数据预处理方法
3. 想要的请求和想要得到的结果

In [47]:
app = flask.Flask(__name__)
model = None
use_gpu = False

In [48]:
with open( '../data/imagenet_class_index.txt', 'r' ) as f:
    idx2lbl = eval( f.read() )

In [49]:
def load_model():
    global model
    model = resnet50( pretrained=True )
    model.eval()
    
    if use_gpu:
        model.cuda() 

In [50]:
def prepare_image( image, target_size ):
    if image.mode != 'RGB':
        image = image.convert( 'RGB' )
        
    image = transforms.Resize( target_size )( image )
    image = transforms.ToTensor()( image )
    image = transforms.Normalize( mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225] )( image )
    
    image = image[ None ]
    if use_gpu:
        image = image.cuda()
    return Variable( image, volatile = True )

    

开启服务

In [51]:
@app.route( '/predict', methods = [ 'POST' ] )
def predict():
    data = { 'success': False }
    
    if flask.request.files.get( 'image' ):
        image = flask.request.files[ 'image' ].read()
        image = Image.open( io.BytesIO(image) )
        
        image = prepare_image( image, target_size = ( 224, 224 ) )
        
        predicts = F.softmax( model( image ), dim = 1 )
        results = torch.topk( predicts.cpu().data, k = 3, dim = 1 )         # 可能性最大的k个预测结果
        
        data[ 'predictions' ] = list()
        
        for prob, label in zip( results[0][0], results[1][0] ):
            label_name = idx2lbl[ label ]
            r = { 'label': label_name, 'probability': float( prob ) }
            data[ 'predictions' ].append( r )
            
        data[ 'success' ] = True
        
        
    return flask.jasonify( data )

In [52]:
load_model()
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
C:\Users\EC319\AppData\Local\Temp\ipykernel_11636\3025148707.py:12: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable( image, volatile = True )
[2024-07-21 20:47:02,809] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "D:\anaconda\Lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\anaconda\Lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])